In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import run_full_analysis 

### Define task function

In [ ]:
N_CHAINS = 32
ISO3 = "FRA"
ANALYSIS = "main"

def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        opti_params={'n_searches': 8, 'num_workers': 8, 'warmup_iterations': 0, 'search_iterations': 10000, 'init_method': 'LHS'},
        mcmc_params={'draws': 10000, 'tune': 0, 'cores': N_CHAINS, 'chains': N_CHAINS},
        full_run_params={'samples': 1000, 'burn_in': 5000},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, f"{ANALYSIS}_LHS8_opti10k_4cores")
run_path

In [ ]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
from time import sleep

update_freq = 10
status = runner.s3.get_status()
print(status)

while status in ['INIT', 'LAUNCHING']:
    sleep(update_freq)
    status = runner.s3.get_status()
print(status)

while status in ['RUNNING']:
    sleep(update_freq)
    status = runner.s3.get_status()
print(status)

In [ ]:
from autumn.core.runs import ManagedRun

In [ ]:
mr = ManagedRun(run_path)

In [ ]:
for f in mr.remote.list_contents():
    mr.remote.download(f)